In [ ]:
from __future__ import print_function, division
from sympy import *
import ast
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
#from brokenaxes import brokenaxes

In [ ]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [ ]:
name = "free_energy_def.dat"
tmp = open(name, "r")
tmp=tmp.read()
exec(tmp)

In [ ]:
name = "assign_symbols_final.dat"
tmp = open(name, "r")
tmp=tmp.read()
exec(tmp)

In [ ]:
w_array = np.loadtxt("w_array.dat")

In [ ]:
name = "ML_parameter.dat" #load n_conv, con_dim,kill (killsmall)
tmp = open(name, "r")
tmp=tmp.read()
exec(tmp)
kill=float(kill)
print(n_parameter,conv_dim,n_conv,kill)

In [ ]:
fed_eqn

In [ ]:
#fed_eqn()

In [ ]:
name = "all_parameter.dat"
tmp = open(name, "r")
tmp=tmp.read()
#print(tmp)
tmp=tmp.replace("/varible:0",":")
tmp=tmp.replace("/const_1:0",":")

for i in range(100):
    tmp=tmp.replace("_"+str(i),"")
tmp=tmp.replace("[[","")
tmp=tmp.replace("]]","")
tmp=tmp.replace("\n",",")
tmp=tmp.replace(":","\":")
tmp=tmp.replace(",",",\"")
tmp="{\""+tmp+"}"
tmp=tmp.replace(",\"}","}")
#print(tmp)
dict_all=ast.literal_eval(tmp)

In [ ]:
dict_all.keys()

In [ ]:
kill_small=0
for i in list(dict_all.keys()):
    if(np.abs(dict_all[i])<kill and i[0]=="a"):
        dict_all[i]=0
        kill_small+=1
print(len(dict_all.keys()),kill_small)

In [ ]:
input_symbols = fed_eqn().subs(dict_all).free_symbols

In [ ]:
input_str = "["
for i in range(n_parameter):
    input_str+="seps"+str(i)+","
for i in range(n_conv):
    input_str+="sn"+str(i)+","
input_str += "]"
tmp = "input_str="+input_str
exec(tmp)

In [ ]:
input_str

In [ ]:
fd = lambdify(input_str,fed_eqn().subs(dict_all))

In [ ]:
fd

In [ ]:
fed_eqn()

In [ ]:
fed_eqn().subs(dict_all)

In [ ]:
dict_all

In [ ]:
for i in range(n_conv):
    eqn_str = "fd_n"+str(i)+" =lambdify(input_str,diff(fed_eqn().subs(dict_all),sn"+str(i)+"))"
    #print(eqn_str)
    exec(eqn_str)

In [ ]:
fd

In [ ]:
fd_n0

In [ ]:
data_file = '../LJ_data_wall/data_all/'
f=np.loadtxt(data_file+"MC_parameter.dat")
L = float(f[0])
dx = float(f[1])
N = int(L/dx)
print(N,L,dx)

In [ ]:
n_array = np.zeros([n_conv,N])

In [ ]:
#@jit
def conv (rho,w,flip):
    LLw = int((conv_dim-1)/2)
    #print(LLw)
    rho_pad=np.copy(rho)
    rho_tile=np.tile(rho_pad,3)
    rho_pad=rho_tile[N-LLw:2*N+LLw]
    if(flip==1):
        n = np.correlate(rho_pad,np.flip(w))
    else:
        n = np.correlate(rho_pad,w)
    return n*dx

In [ ]:
rho_test=np.full(N,0.1)
n0=conv(rho_test,w_array[0],1)

In [ ]:
n0.shape

In [ ]:
x=np.linspace(-conv_dim*dx/2,conv_dim*dx/2,conv_dim)
for i in range (n_conv):
    plt.plot(x,w_array[i],linewidth=3,label=r"$\omega_{"+str(i)+"}$")
plt.legend(fontsize=20,loc="lower right")
plt.xlabel(r"$x/\sigma$",fontsize=25)
plt.ylabel(r"$\omega$",fontsize=25)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
#plt.xlim([-4,4])
#plt.ylim([np.min(w_array)-0.05,0.8])
#plt.text(-12.5,1.2,r"$(b)$",fontsize=25)
plt.savefig("wlj_tail.eps",format="eps",bbox_inches='tight')

## one parameter

In [ ]:
def w_FMT(eps):
    #d=cal_eff_diameter (eps)
    R = 1.0/2
    k=np.linspace(0,N//2,N//2+1)*2*np.pi/L
    w0=2*np.cos(k*R)/2
    k[0]=1 #keep notebook shutup
    w1=2*np.sin(k*R)/k
    w1[0]=2*R
    return w0,w1;

def cal_n(rho,w):
    return np.fft.irfft(np.fft.rfft(rho)*w)

def cal_c1_FMT(rho,eps=0):
    w0,w1 = w_FMT(eps)
    n0=cal_n(rho,w0)
    n1=cal_n(rho,w1)
    F0=-np.log(1-n1)
    F1=n0/(1-n1)
    #print(np.min(1-n1))
    return cal_n(F0,w0)+cal_n(F1,w1),-n0*np.log(1-n1)
    #return np.zeros(len(rho)),np.zeros(len(rho))
    

In [ ]:
#@jit
def cal_df(rho,w,eps):
    
    n_array = np.zeros([n_conv,N])
    n1=np.zeros(N)
    tmp=""
    for i in range(n_conv):
        n_array[i]=conv(rho,w[i],1)

    #print(n_array)
    input_str="("+str(eps)+","
    for i in range(n_conv):
        input_str+="n_array["+str(i)+"]"
        if(i!=n_conv-1):
            input_str+=","
    input_str += ")"
    #print(input_str)
    c1=0
    for i in range(n_conv):
        string = "fd_n"+str(i)+input_str
        fn=eval(string)
        c1+=conv(fn,w[i],0)
    #print(c1[0])
    fed = "fd"+input_str
    #print(fed)
    fed = eval(fed)
    #print(fed)
    
    #return c1,fed
    return np.float(eps)*c1+cal_c1_FMT(rho)[0],eps*fed+cal_c1_FMT(rho)[1]

In [ ]:
def eos(rho0,w,eps):
    rho=np.full(N,rho0)
    c1,f=cal_df(rho,w,eps)
    p=f-rho*c1
    return -p[0]+rho0

In [ ]:
def cal_P_ML(eps):
    rho0_array=np.linspace(0,0.85,20)
    P_ML=np.zeros(len(rho0_array))
    for i in range (len(P_ML)):
        P_ML[i]=eos(rho0_array[i],w_array,eps)
    return rho0_array,P_ML

In [ ]:
c=['g','r','b','k']
eps_array=[1.0,1.5,2.5,3.5]
for i in range (4):
    eps=eps_array[i]
    #c=np.random.rand(3)
    #print(c)
    P_MC = np.loadtxt("../LJ_pressure/P_rho_eps_"+str(eps)+".dat")
    plt.plot(P_MC[:,1],P_MC[:,0],"--",color=c[i],label=r"MC, $\epsilon$ ="+str(eps),lw=3)
    P_ML=cal_P_ML(eps)
    plt.plot(P_ML[0],P_ML[1],"o",markerfacecolor="None",
         markeredgecolor=c[i], markeredgewidth=2,label=r"ML",lw=2)
plt.ylim([-0.1,6])    
plt.legend(fontsize=14,loc='upper left')
plt.ylabel(r"$P\sigma$",fontsize=25)
plt.xlabel(r"$\rho\sigma$",fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.text(-0.2,6/8*9,r"$(a)$",fontsize=25)
plt.savefig("PLJ_tail.eps",format="eps",bbox_inches='tight')

In [ ]:
def mu_FMT (rho):
    R=0.5
    return (2*R*rho)/(1 - 2*R*rho) - np.log(1 - 2*R*rho)+np.log(rho)

In [ ]:
rho0_array=np.linspace(0.01,0.9,100)
mu_ML=np.zeros(len(rho0_array))
for i in range (len(mu_ML)):
    mu_ML[i]=(cal_df(np.full(N,rho0_array[i]),w_array,eps)[0])[0]+np.log(rho0_array[i])

In [ ]:
#@jit
def picard(rho,Vext,w,eps,alpha=10**-3,mu=0):
    c1 = cal_df(rho,w,eps)[0]
    rho_new = np.exp(-c1-Vext)
    #mu=np.log(np.sum(rho)/np.sum(rho_new))
    rho_new *= np.exp(mu)
    error=np.mean((rho_new-rho)**2)
    rho=(1-alpha)*rho+alpha*rho_new
    
    return rho,error

In [ ]:
def c1_MF(rho,eps):
    r=np.linspace(-L/2,L/2,N+1)
    #print(type(r),type(eps))
    U_base=4*eps*(1/r**12-1/r**6)
    
    
    U_base[np.where(np.abs(r)<1)]=0
    LLw = int(N//2)
    #print(LLw)
    rho_pad=np.copy(rho)
    rho_tile=np.tile(rho_pad,3)
    rho_pad=rho_tile[N-LLw:2*N+LLw]
    n = np.correlate(rho_pad,U_base)
    #print(n.shape,U_base.shape)
    return n*dx

#@jit
def cal_df_MF(rho,eps):

    c1=c1_MF(rho,eps)
    
    return c1+cal_c1_FMT(rho,eps)[0]

#@jit
def picard_MF(rho,Vext,eps,alpha=10**-3,mu=0):
    eps=float(eps)
    c1 = cal_df_MF(rho,eps)
    rho_new = np.exp(-c1-Vext)
    rho_new *= np.exp(mu)
    error=np.mean((rho_new-rho)**2)
    rho=(1-alpha)*rho+alpha*rho_new
    
    return rho,error

In [ ]:

f = open(data_file+'/MC_inform.dat', 'r')
MC_inform = f.read().splitlines()
for i in range(len(MC_inform)):
    #print(MC_inform[i])
    MC_inform[i]=MC_inform[i].split("\t")
f.close()
#MC_inform

In [ ]:

#i=np.random.randint(0,32)
i=178
print(MC_inform[i])
print("i=",i)
Vext = np.loadtxt(data_file+'/Vext_'+str(i)+'.dat')
rho_test = np.loadtxt(data_file+'/rho_'+str(i)+'.dat')
mu = np.log(float(MC_inform[i][2]))
eps = float(MC_inform[i][1])
print(i,eps,mu)
#mu=0.22182268
#eps=2

alpha=10**-3
rho=np.full(N,np.mean(rho_test))
error_array=[]
threshold=10**-6
for i in range(100000):
    rho,error=picard(rho,Vext,w_array,eps,alpha,mu)
    error_array+=[error]
    if(i%1000==0):
        print(error)
    if(error<threshold):
        break
        
rho_MF=np.full(N,0.1)
error_array=[]
threshold=10**-6
for i in range(1000000):
    rho_MF,error=picard_MF(rho_MF,Vext,eps,alpha,mu)
    error_array+=[error]
    if(i%1000==0):
        print(error)
    if(error<threshold):
        break

In [ ]:
plt.plot(rho)
plt.plot(rho_test)


In [ ]:
data_file="../LJ_data_wall/data_all"
f = open(data_file+'/MC_inform.dat', 'r')
MC_inform = f.read().splitlines()
for i in range(len(MC_inform)):
    #print(MC_inform[i])
    MC_inform[i]=MC_inform[i].split("\t")
f.close()
MC_inform

In [ ]:

i=180
#rho_wall = np.loadtxt('LJ_data_fuzzy/LJ_python_wall/wall_mu_2_eps_2_MC.dat')
#Vext_wall = np.loadtxt('LJ_data_fuzzy/LJ_python_wall/wall_Vext.dat')
Vext = np.loadtxt(data_file+'/Vext_'+str(i)+'.dat')
rho_test = np.loadtxt(data_file+'/rho_'+str(i)+'.dat')

mu = np.log(float(MC_inform[i][2]))
eps = np.float(MC_inform[i][1])
print(mu,eps)
#mu=0.22182268
#eps=2
print(i,eps,np.exp(mu))
alpha=10**-3
rho=np.full(N,np.mean(rho_test ))
error_array=[]
threshold=10**-6
for i in range(100000):
    rho,error=picard(rho,Vext,w_array,eps,alpha,mu)
    error_array+=[error]
    if(i%1000==0):
        print(error)
    if(error<threshold):
        break
        
rho_MF=np.full(N,0.1)
error_array=[]
threshold=10**-6
for i in range(1000000):
    rho_MF,error=picard_MF(rho_MF,Vext,eps,alpha,mu)
    error_array+=[error]
    if(i%1000==0):
        print(error)
    if(error<threshold):
        break

In [ ]:
x=np.linspace(0,L-dx,N)
grid = plt.GridSpec(4, 1, wspace=0.4, hspace=0.5)
plt.subplot(grid[0, 0])

plt.text(0.1,2,r"$\infty$",fontsize=20)
plt.plot(x,Vext,"k-",lw=3)
plt.xlim([0.02,15])
plt.ylim([-4,10])
plt.text(-3,15,r"$(b)$",fontsize=25)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.yticks(fontsize=15)


plt.ylabel(r"$V^{\mbox{\large  ext}}$",fontsize=25)

plt.subplot(grid[1:, 0])
plt.plot(x,rho_test,"k-",lw=3,label="MC")
plt.plot(x,rho,"b--",lw=3,ms=2,label="ML")
plt.xlim([0.02,15])
plt.legend(fontsize=20,loc="upper right")
plt.ylabel(r"$\rho\sigma$",fontsize=25)
plt.xlabel(r"$x/\sigma$",fontsize=25)
#plt.title(r"$\rho_0\sigma=0.43$",fontsize=25)
plt.xticks(fontsize=20)
plt.yticks([0,0.4,0.8,1.2],fontsize=20)
plt.savefig("rho_LJ_tail_inside.eps",format="eps",bbox_inches='tight')

In [ ]:
x=np.linspace(0,L-dx,N)
grid = plt.GridSpec(4, 1, wspace=0.4, hspace=0.5)
plt.subplot(grid[0, 0])

plt.text(0.1,2,r"$\infty$",fontsize=20)
plt.plot(x,Vext,"k-",lw=3)
plt.xlim([0.02,15])
plt.ylim([-4,10])
plt.text(-3,15,r"$(b)$",fontsize=25)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.yticks(fontsize=15)


plt.ylabel(r"$V^{\mbox{\large  ext}}$",fontsize=25)

plt.subplot(grid[1:, 0])
plt.plot(x,rho_test,"k-",lw=3,label="MC")
plt.plot(x,rho,"b--",lw=3,ms=2,label="ML")
plt.plot(x,rho_MF,"r-.",lw=3,ms=2,label="MF")

plt.xlim([0.02,15])
plt.legend(fontsize=20,loc="upper right", bbox_to_anchor=(0.99, 0.99))
plt.ylabel(r"$\rho\sigma$",fontsize=25)
plt.xlabel(r"$x/\sigma$",fontsize=25)
#plt.title(r"$\rho_0\sigma=0.43$",fontsize=25)
plt.xticks(fontsize=20)
plt.yticks([0,0.4,0.8,1.2],fontsize=20)
plt.savefig("rho_LJ_tail_inside_w_MF.eps",format="eps",bbox_inches='tight')

In [ ]:
x=np.linspace(0,L-dx,N)
plt.plot(x,rho_test,"k-",lw=3,label="MC")
plt.plot(x,rho,"b-.",lw=3,ms=2,label="ML")
plt.legend(fontsize=35,loc="upper left")
plt.ylabel(r"$\rho\sigma$",fontsize=25)
plt.xlabel(r"$x(\sigma)$",fontsize=25)
#plt.title(r"$\rho_0\sigma=0.43$",fontsize=25)
#plt.savefig("rho_LJ_tail_inside.eps",format="eps",bbox_inches='tight')

In [ ]:
data_file="../LJ_data_wall_test/data_0"
f = open(data_file+'/MC_inform.dat', 'r')
MC_inform = f.read().splitlines()
for i in range(len(MC_inform)):
    #print(MC_inform[i])
    MC_inform[i]=MC_inform[i].split("\t")
f.close()
MC_inform

In [ ]:
i= 2
#rho_wall = np.loadtxt('LJ_data_fuzzy/LJ_python_wall/wall_mu_2_eps_2_MC.dat')
#Vext_wall = np.loadtxt('LJ_data_fuzzy/LJ_python_wall/wall_Vext.dat')
Vext = np.loadtxt(data_file+'/Vext_'+str(i)+'.dat')
rho_test = np.loadtxt(data_file+'/rho_'+str(i)+'.dat')

mu = np.log(float(MC_inform[i][2]))
eps = np.float(MC_inform[i][1])
print(type(eps))
#mu=0.22182268
#eps=2
print(i,eps,np.exp(mu))
alpha=10**-3
rho=np.full(N,np.mean(rho_test))
error_array=[]
threshold=10**-6
for i in range(1000000):
    rho,error=picard(rho,Vext,w_array,eps,alpha,mu)
    error_array+=[error]
    if(i%1000==0):
        print(error)
    if(error<threshold):
        break
        
rho_MF=np.full(N,0.1)
error_array=[]
threshold=10**-6
for i in range(1000000):
    rho_MF,error=picard_MF(rho_MF,Vext,eps,alpha,mu)
    error_array+=[error]
    if(i%1000==0):
        print(error)
    if(error<threshold):
        break

In [ ]:
x=np.linspace(0,L-dx,N)
grid = plt.GridSpec(4, 1, wspace=0.4, hspace=0.5)
plt.subplot(grid[0, 0])

plt.text(0.15,2,r"$\infty$",fontsize=20)
plt.plot(x,Vext,"k-",lw=3)
plt.ylim([-4,10])
plt.xlim([0.1,10])
plt.text(-3,15,r"$(c)$",fontsize=25)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.yticks(fontsize=20)
plt.yticks(fontsize=20)


plt.ylabel(r"$V^{\mbox{\large  ext}}$",fontsize=25)

plt.subplot(grid[1:, 0])
plt.plot(x,rho_test,"k-",lw=3,label="MC")
plt.plot(x,rho,"b--",lw=3,ms=2,label="ML")
#plt.legend(fontsize=12,loc="lower right")
plt.ylabel(r"$\rho\sigma$",fontsize=25)
plt.xlabel(r"$x/\sigma$",fontsize=25)
plt.xlim([0.1,10])
plt.ylim([-0.1,2.4])
plt.xticks(fontsize=20)
plt.yticks([0,1,2],fontsize=20)
#plt.title(r"$\rho_0\sigma=0.43$",fontsize=25)
plt.savefig("rho_LJ_tail_outside.eps",format="eps",bbox_inches='tight')

In [ ]:
x=np.linspace(0,L-dx,N)
grid = plt.GridSpec(4, 1, wspace=0.4, hspace=0.5)
plt.subplot(grid[0, 0])

plt.text(0.15,2,r"$\infty$",fontsize=20)
plt.plot(x,Vext,"k-",lw=3)
plt.ylim([-4,10])
plt.xlim([0.1,10])
plt.text(-3,15,r"$(c)$",fontsize=25)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.yticks(fontsize=20)
plt.yticks(fontsize=20)


plt.ylabel(r"$V^{\mbox{\large  ext}}$",fontsize=25)

plt.subplot(grid[1:, 0])
plt.plot(x,rho_test,"k-",lw=3,label="MC")
plt.plot(x,rho,"b--",lw=3,ms=2,label="ML")
plt.plot(x,rho_MF,"r-.",lw=3,ms=2,label="MF")
#plt.legend(fontsize=12,loc="lower right")
plt.ylabel(r"$\rho\sigma$",fontsize=25)
plt.xlabel(r"$x/\sigma$",fontsize=25)
plt.xlim([0.1,10])
plt.ylim([-0.1,2.4])
plt.xticks(fontsize=20)
plt.yticks([0,1,2],fontsize=20)
#plt.title(r"$\rho_0\sigma=0.43$",fontsize=25)
plt.savefig("rho_LJ_tail_outside_w_MF.eps",format="eps",bbox_inches='tight')